## 1. Project goal
My personal target in this project is to identify at least one more person than was in the official list of the Enron fraud investigation. I am well aware that statistical methods will not be sufficient as prove in a fraud case, so the name will not be published - however, i am interested in how this very simple information like financial information and email network information as given here, can uncover connections.
Out of personal interest, i also want to see how far I can come with non-Neural-Network methods. As long as there is no PCA used, the non-NN-methods frequently are able to give insight.



### Some remarks
I am using Python 3.6

I needed to change all print commands to include brackets, and more important, i needed to change the pickle dump() and read() functions to use rb instead of r and wb instead of w.

Should the 2.7 version of Udacity not be able to execute, please try in Python 3.6 and use my modified tester function.

## Data investigation
The code for the investigation can be found in 01_Data_investigation.py

### outlier detection
Outlier detection is the central goal of this project - any unusual salary would be "interesting". However, in the first phase, when checking the data for the first time, one point was really special:
<img src="Outlier detection.png">
This picture was saved from the console during the investigation phase. Checking the data, the point to the top right was a "Total" that really shouldn't be in the dataset. It was removed.

### other data properties
This is a very small dataset. We have a total of 145 rows. 18 of those are POI; so the dataset is highly unbalanced, too. 
The data contains also a lot of sparsity. I did a count of NAN values:
```
print(data_df[data_df=="NaN"].count(axis=0))
bonus                         64
deferral_payments            107
deferred_income               97
director_fees                129
email_address                 34
exercised_stock_options       44
expenses                      51
from_messages                 59
from_poi_to_this_person       59
from_this_person_to_poi       59
loan_advances                142
long_term_incentive           80
other                         53
poi                            0
restricted_stock              36
restricted_stock_deferred    128
salary                        51
shared_receipt_with_poi       59
to_messages                   59
total_payments                21
total_stock_value             20
```
Only one field has no NaN values. It is the one made manually by Katie, to have labels for the data: POI.


## 2. Feature selection
#### how did i pick them
The feature selection was the main part of the project - i did it all the way right up to the end. You will find three main "batches" of feature selection:
1. In the first attempt, use them all and choose some algorithms.
2. Then engineer my own features and see how the algorithms behave.
3. Finally, combine the best of feature selection and algorithm selection for the result.

In a first attempt, i selected all financial information available (after all, why committing fraud if you have no interest doing so?); and relational information from the email data: Emails sent to and from POI; and emails sent to this person with an POI in copy (indicating probably that the sender thinks there is a relation)

Next I made a correlation matrix to see if there is redundant data that could be combined or removed.

The detailed list is as follows

    features_list = ['poi', 
        'salary', 'bonus', 'director_fees',  'expenses',
        'loan_advances', 'long_term_incentive', 'other', 
        'total_payments',
        
        'deferral_payments', 'deferred_income',
        
        'exercised_stock_options', 'total_stock_value', 
        'restricted_stock', 'restricted_stock_deferred',
        
        'shared_receipt_with_poi', 'to_messages', 'from_messages', 
        'from_poi_to_this_person', 'from_this_person_to_poi']

<img src="Correlation_matrix.png">

It can be seen in the correlation matrix that nearly all features have a correlation to the poi feature, which is good. 
However, the first three stock option features (excercised stock options, total stock value, restricted stock) are so deeply correlated that it may be interesting to remove some.
The email features (last five lines) are also very much in line with each other.

### what about sparse data
Coming back to the sparsity problem: I was experimenting with leaving those features out of the estimator, but the results have not been better, so they stay in. I checked with the final algorithm selected in the last part of this project, if a removal of those data fields has an influence on the performance.
``` 
results for leaving some sparse data out
All features, SVC: F1=0.488  # This is the winner
w/o loan_advances, SVC: F1=0.487
w/o restricted_stock_deferred, SVC: F1=0.479
w/o director_fees, SVC: F1=0.481
```
So, sparse the data may be, but information it carries nevertheless. Inside the stack i leave them!

Astonishingly, some of the not-so-sparse data contained less information. When i removed those three features from the stack, the algorithm performance actually boosted.
```
w/o 'exercised_stock_options', SVC: F1= 0.511
w/o 'shared_receipt_with_poi', SVC: F1=0.515
w/o 'to_messages', SVC: F1=0519
```
These are the features i finally removed from the list. Note, this final feature removal was done right at the end, after the algorithm was selected.

Before selecting the final features, i did a first try with some models

## 3. Model selection
I was testing AdaBoost, SVC, Random Forest Classifier and Decision Tree Classifier.
All four of them are good classifiers. Only the SVC needs some preprocessing, which is also nice: The more i can do with the original data, the less i am in danger of accidentally introducing a failure.

From here i tried various strategies:
1. Apply the algorithms with a minimum of preprocessing
2. Generate additional random features and let PCA / KBest choose the best ones
3. Choose from the features with PCA and KBest before applying the Algorithms
4. Add only the best few of the additional random features, use PCA/KBest to choose
5. Evaluate all strategies and choose the best.

After several unsuccessful tries, I changed my test set up so that it was *very* similar to the Udacity tester code, in order to yield good results there. I load the same deprecated StratifiedShuffleSplit function in order to be able to bring the folds up to a thousand in the end. During the experimentation phase, however, i kept the folds down to 10 for the calculation speed. As Udacity requires to optimize on Precision and Recall, i use the F1 score as metric for the evaluation.

#### F1 score results in the first try
* AdaBoost F1: 0.25
* Decision Tree F1: 0.24
* Random Forest F1: 0.30
* SVC F1: 0.42


## 4. Parameter selection: Turning knobs
There are two kinds of hyperparameters: One has to be found by search, the other one by knowledge. One example for knowledge-based parameters is the class_weight. Three out of my four models had a class_weight parameter that i set to "balanced" in order to equalize the imbalance in the training data. I would even include the choice of scaler into this category: SKLearn offers, like, tens of scalers. The SVC and PCA algorithms both need roughly gaussian input, so the standardScaler was the right choice here.

Other parameters have to be searched. The number of estimators in random forest or Adaboost; or the C and Gamma values in the SVC are examples for this. The principle of search was to refine the grid in three or four steps, until the algorithms were well adapted to the data, and equally important, to the scorer. This is the only place where you can adapt the shape of your function to the needs of your scorer - if it will have a better accuracy or a better recall is decided here.
If this step is done wrong, we have an overfit estimator that won't give me any prediction. 

Example: If i optimized the SVC on accuracy here, it would be like 90% accurate by just ignoring that there are POI's. It is for this very reason that i use the F1 score all along the code to fit the estimators.

In detail, an example for one value of the SVC:

First step
```
{'C' : [1, 2, 5, 1e1, 2e1, 5e1, 1e2, 2e2, 5e2,
               1e3, 2e3, 5e3, 5e4, 5e5, 5e6 ]}
```
Chosen: 5e2. Second step
```
{'C' : [300, 400, 500, 600, 700, 800]}
```
Chosen: 600. Third step
```
{'C' : [550, 580, 600, 620, 640]}
```
Chosen: 620 --> And stop.

I was thinking that this "turning knobs" step would be repetitive for every machine learning task and wondered that there is no "hyperfit" function in SKlearn.


#### F1 score for different kernels of svc
What's more, the SVC algorithm comes with different "kernels"; that are nearly algorithms on their own, really. I tried them all with the above mentioned grid and found that the sigmoid function was superior to the others.

* RBF 0.42
* Sigmoid 0.64
* linear - no solution
* poly (grade 3) 0.25

After the sigmoid variant proved to be so superior, the values for C and gamma were refined by a variant of successive approximation (done manually). The F1 score looked extremely promising with 0.64 - The udacity tester code agreed by showing a precision of 0.33 and a recall of 0.73.

## Featuring II
There is still one task left: See if we can improve the result by feature engineering. Generally, there seem to be three strategies out in the wild:

* Use inside knowledge to make the right feature at the right scale 
* generate loads of random features and choose the best with some kind of algorithm - Lasso, KBest, PCA
* avoid feature engineering and use neural networks instead. 

Neural networks are not yet an option for me, and i have no insider knowledge about Enron at all.
So i will try some experimental random feature engineering here; then I employ KBest and PCA to define a cut-off, and then i let myself be surprised by the (hopefully) one or two valuable new features that i find in the mass. Another expectation is to see; through higher dimensionality, better separated grouping. 

One drawback is that too many dimensions compared to data rows can easily break the algorithms; 

a second drawback that really gave me a bad feeling about this is that if you generate calculated features from sparse data, the result is a lot more sparse. That is especially true for division features, which is sad as a number divided by another one usually depicts a percentage, and those are often helpful.

Nevertheless, i wanted to try this experiment, so here is what I did:

I multiplied every column with every column. 
I subtracted every column from every column. 
I divided every column by every column. 
The resulting 1200 features  were fed into SelectKBest with the f_classif algorithm; and they were scaled and put into a PCA. 

I trained the four Algorithms of above, in a for-loop, on anything between 1 and 100 features coming out of the two feature selection methods. 
I expected some kind of e-curve approximation, and so was I quite surprised to see no steady decline or rise in my f1-value, but a quite wild zig-zag; and a nearly unreadable graphic in the end.

<img src="Algo performance F1.png">


The top line is interesting: The graphic ends well below 0.6; however, with the original features and without much in terms of preprocessing, we were already better than that. So in total, this was not the best idea.

The other three colors represent Adaboost (yellow), random forest (green/grey) and decision tree (blue); each time the dark variant for PCA and the light variant for KBest. They also represent my (before) missing experience: Those three algorithms are very well capable of selecting the features themselves; interfering with them is not a good idea as feature selection is part of what makes them special compared to the others. For me that's a win! I save that as "experience".

Finally, the two reddish lines depict the Support Vector Classifier. I was using the same parameters that have already been tuned before - and interestingly, it seems to work very well together with PCA. That might be because PCA delivers gaussian values, and for the SVC the ideal input is exactly that.

To get any insight at all, I tried the same with the original dataset.

<img src="Algo performance F1 original data.png">

First we note that the performance is much less random with the original features. Second, we can see that the peak performance is better than the peak performance with the random additional features, nearly 0.6 F1 score. Third, we can see that the two favorites both outperform with PCA and not with K-Best; which is the opposite result than above. Indeed, K-Best seems to work best on the right side of the scale: At the point where it actually does *not select* anymore.

Well - this exercise was nice, but in the beginning i did this to see if there is one calculated feature that can actually improve the original score.

### Choosing additional features
I executed a little code:

    kb = SelectKBest(k=5)
    kb.fit(lots_X, lots_Y)
    for i, choice in enumerate(kb.get_support()):
    if choice:
        print(features_list_all[i])

And indeed, there were some important features; and they are exclusively multiplication features.
```
deferred_income * exercised_stock_options
deferred_income * total_stock_value
exercised_stock_options * deferral_payments
exercised_stock_options * deferred_income
total_stock_value * deferred_income
```
Two of those five chosen features are actually double! This might be in parts the explanation for the erratic behavior of the algorithms - if they get "additional" features that really don't contain more information, it might harm the performance.

I continue by adding those three features to the original feature vector and see how the algorithms behave.
As AdaBoost and Random Forest didn't compare well to the other two, they are excluded from now on for speed and readability.

<img src="Algo performance F1 three Features.png">

This last graph is much better for the eyes :) After adding of the three additional features, both algorithms seem to peak around five features; which is earlier than before. As before, the decision tree gets better the less K-Best is involved - after all, the decision tree is quite capable of choosing features himself. Most interesting is the peak at six features for the decision tree, with PCA selected features. Why does it go down afterwards? I don't know.

**At this point i abandoned the feature selection by PCA and K-Best; as the results were not better than without before**




## 5. final validation
Having gained the experience now, i was re-fitting the same four algorithms on the original data again, now with the best parameters and features available. Also, i found out that i forgot to set "class_weight" to balanced in the first try. Finally having learned now that i can set the base estimator of AdaBoost to something i like, i did use the most successful variant of a Decision Tree in Adaboost. Then i put the StratifiedShuffleSplit to 1000 for the GridSearch; so that it was exactly like in the udacity tester code and retuned all estimators. 

**why this extra-validation?**
This is a necessary step: Without the validation from the Udacity tester code, i can never be sure that i did not overfit my estimator to the wrong metric. I was choosing features and training the algorithms all the way long with a simple F1 scorer and a simplified (compared to the Udacity) stratifiedShuffleSplit, in order to save time. It can of course happen that i was not finding the best parameters or i was overfitting somewhere. So this step here is necessary to confirm that what i did is good the way Udacity is measuring.

The validation was done each time with the Udacity tester code and recorded in a table. Marked in red were all algorithms that didn't fulfill the Udacity requirements of at least 0.3 Precision and Recall; marked in green are the top three results in each column.

Nearly as afterthought, i was letting the other algorithms use the DecisionTree choice of features, just to see if they would behave better or not.
<img src="Final performance table.png">

Now, having the confirmation that in each case the SVC behaves best, i was refitting it on the best feature vector i could make. 
This is the feature list:
```
['poi',
 'salary', 'director_fees', 'bonus', 'expenses', 
 'loan_advances', 'long_term_incentive', 'other',
 'total_payments', 'deferral_payments', 'deferred_income',
 'total_stock_value', 'restricted_stock', 'restricted_stock_deferred',
 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi',
 
 'deferred_income_x_exercised_stock_options', 
 'deferred_income_x_total_stock_value',          
 'exercised_stock_options_x_deferral_payments']
```

Those last three features are the ones derived from the experiment with the 1200 artificial features.
On this list, with an SVC *without PCA*, using the sigmoid function, balanced class weight, C=600, gamma = 0.01, i achieved with the Udacity tester code:

* Accuracy: 0.77487       
* Precision: 0.34851      
* Recall: 0.79200 
* F1: 0.48403
* F2: 0.63133

As i didn't optimize for the accuracy but for the F1 score, my top result for it is 0.77: that means 0.10 less than the benchmark for this exercise. However, i am quite happy with a recall rate of 0.79 and a precision of 0.35 - those results are more important than the pure accuracy, given that this work could point it's finger at certain persons.

## 6. Two evaluation metrics
Personally i think that none of this is going to be usable for a new case: As it stands, the precision of 0.34 indicates that my algorithm points out two non-POI's for every POI marked in the original data. A deep investigation would be necessary now to see if those two extra are a really great result of my work and they need to be investigated by the justice system, or if the data is just not good enough and i point my finger to a lot of innocents.

Similarly, a recall rate of 79% means that my algorithm finds 79% of the POI's in the dataset. Having 18 POI's means it finds 14, and 4 get away. At least the recall rate should be beyond 90% before doing any serious justice action on this. However, i tried to train my algorithm to that metric first, with the result that both accuracy and precision were going so far down that it wasn't usable.

(Wikipedia has a nice explanation) Those two metrics show that it is not so easy to identify fraud from email and financial data only. It would be interesting to see how well a neural net performs here - whereas, it is somewhere between difficult and impossible to find the reasons for a decision in the net; which is of prime importance in the justice. An SVC with a sigmoid decision function is as close as it gets to a Neural Net without actually setting one up.

## Conclusion:
Having tried and tested lots values in the hyperparameter space, having played around a lot with the features, this project showed me a few interesting things:

* I can only increase recall and precision at the same time, when i find a much better feature in the original bulk of the e-mails. This is a work i will not do in the frame of this project.
* I just didn't want to stop as i had the feeling that it could be yet a little bit better. And then some.
* Behind each of the sklearn "algo's" there is a lot of things yet to learn - using the decision tree for feature selection was interesting, also using K-Best, or observing the behavior of PCA generated features. Especially PCA could be quite powerful if one feels the need to reduce a big feature space. Here, with 20 features, it was not necessary.
* The coding for this project was... interesting. As i had to go forward and backward in the code, constantly changing parts of it and in the go deleting traces of before, the poi_id.py now contains just the traces of the best approach. There is poi_id_mass_test that contains a lot more traces that stem in part from the data investigation and in part from the search for good additional features. Most of the code there is commented, because i was activating or deactivating parts of it as needed.
* The one thing i did not do was writing a scorer of my own. Fitting to the question of Udacity, i would make a scorer that has a big penalty function for Precision or Recall below 0.3; and as soon as it is above 0.3, an extra bonus for each additional point of accuracy. That could actually be interesting, having a 5 minutes video during the course about how to do that.

It was definitely interesting and leaves me questioning two things:

* Natural data is dirty. How can we do things like palantir's predictive policing or like COMPAS (used for deciding prison sentences), without putting lots of innocents in risk of being mis-estimated?
* And more personally: how i would fare if attacking the one or other Kaggle dataset - I will see.